In [1]:
import pandas as pd
from unidecode import unidecode
from sys import exit

def trim_overspace(text):
	parts = filter(lambda x: len(x) > 0,text.split(" "))
	return " ".join(parts)

def normalize_text(text):
    #.replace("'"," ") alguns municipios tem ' e as macros maravilhosas do excel vão falhar
	x = str(text).replace(".","").replace("\n","").replace(",","").upper()
	x = trim_overspace(x)
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_labels(text):
	x = str(text).replace("'"," ").replace(".","").replace("\n","").replace(",","").lower()
	x = trim_overspace(x).replace(" ","_")
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_number(num,cast=int,error='fill',fill='0'):
	try:
		return cast(num)
	except ValueError:
		if error == 'raise':
			raise Exception(ValueError)
		elif error == 'fill':
			return normalize_number(fill,cast,'raise')

In [2]:
obitos_raw = pd.read_excel('novos_obitos_raw.xlsx')

In [3]:
obitos_raw.columns = [ normalize_labels(x) for x in obitos_raw.columns]
obitos_raw['nome_paciente'] = obitos_raw['nome_paciente'].apply(normalize_text)
obitos_raw['idade'] = obitos_raw['idade'].apply(int)
obitos_raw['municipio_residencia'] = obitos_raw['municipio_residencia'].apply(normalize_text)
obitos_raw['municipio_atendimento'] = obitos_raw['municipio_atendimento'].apply(normalize_text)

In [4]:
obitos_raw.dtypes

data_notificacao          datetime64[ns]
id_ficha                           int64
nome_paciente                     object
sexo                              object
idade                              int64
municipio_residencia              object
municipio_atendimento             object
rs                                object
exame                             object
data_diagnostico          datetime64[ns]
data_primeiro_sintomas    datetime64[ns]
data_cura_obito           datetime64[ns]
dtype: object

In [5]:
obitos_raw['hash'] = obitos_raw.apply(lambda row: (row['nome_paciente'] + str(row['idade']) + row['municipio_residencia']).replace(" ",""), axis=1)
obitos_raw['hash_less'] = obitos_raw.apply(lambda row: (row['nome_paciente'] + str(row['idade']-1) + row['municipio_residencia']).replace(" ",""), axis=1)
obitos_raw['hash_more'] = obitos_raw.apply(lambda row: (row['nome_paciente'] + str(row['idade']+1) + row['municipio_residencia']).replace(" ",""), axis=1)

In [6]:
obitos_raw = obitos_raw.sort_values(by='nome_paciente')

In [7]:
print(f"obitos novos: {obitos_raw.shape[0]}")
print(f"obitos novos duplicados: {obitos_raw.loc[obitos_raw.duplicated(subset='hash')].shape[0]}")
obitos_raw = obitos_raw.drop_duplicates(subset='hash')
print(f"obitos novos sem duplicados: {obitos_raw.shape[0]}")

obitos novos: 96
obitos novos duplicados: 0
obitos novos sem duplicados: 96


In [8]:
obitos_confirmados = pd.read_excel('Casos confirmados PR.xlsx', 'Obitos', usecols='D:I')
obitos_confirmados.columns = [ normalize_labels(x) for x in obitos_confirmados.columns]

In [9]:
obitos_confirmados['nome'] = obitos_confirmados['nome'].apply(normalize_text)
obitos_confirmados['idade'] = obitos_confirmados['idade'].apply(normalize_number)
obitos_confirmados['municipio'] = obitos_confirmados['municipio'].apply(normalize_text)
obitos_confirmados['hash'] = obitos_confirmados.apply(lambda row: (row['nome'] + str(row['idade']) + row['municipio']).replace(" ",""), axis=1)
obitos_confirmados.dtypes

nome                     object
sexo                     object
idade                     int64
municipio                object
rs                      float64
data_do_obito    datetime64[ns]
hash                     object
dtype: object

In [10]:
index_obitos_duplicados = obitos_raw.loc[obitos_raw['hash'].isin(obitos_confirmados['hash'])].index.to_list()
print(f"obitos já em obitos com a mesma idade: {len(index_obitos_duplicados)}")
index_obitos_duplicados_idade_less = obitos_raw.loc[obitos_raw['hash_less'].isin(obitos_confirmados['hash'])].index.to_list()
print(f"obitos já em obitos com a idade - 1: {len(index_obitos_duplicados_idade_less)}")
index_obitos_duplicados_idade_more = obitos_raw.loc[obitos_raw['hash_more'].isin(obitos_confirmados['hash'])].index.to_list()
print(f"obitos já em obitos com a idade + 1: {len(index_obitos_duplicados_idade_more)}")
print(f"dentre os quais {len(set(index_obitos_duplicados_idade_more).intersection(index_obitos_duplicados)) + len(set(index_obitos_duplicados_idade_less).intersection(index_obitos_duplicados))} são obitos em comum, o que leva a crer que estão duplicados na planilha já com idade a mais ou idade a menos")
index_duplicados = list(set(index_obitos_duplicados + index_obitos_duplicados_idade_less + index_obitos_duplicados_idade_more))
print(f"sendo assim, {len(index_duplicados)} obitos que já se encontram na planilha serão removidos")

obitos já em obitos com a mesma idade: 54
obitos já em obitos com a idade - 1: 8
obitos já em obitos com a idade + 1: 0
dentre os quais 1 são obitos em comum, o que leva a crer que estão duplicados na planilha já com idade a mais ou idade a menos
sendo assim, 61 obitos que já se encontram na planilha serão removidos


In [11]:
print(f"então, de {len(obitos_raw)} obitos baixados hoje  {len(obitos_raw)-len(index_duplicados)} serão adicionados")
obitos_raw = obitos_raw.drop(index=index_duplicados)
obitos_raw.shape

então, de 96 obitos baixados hoje  35 serão adicionados


(35, 15)

In [12]:
casos_confirmados = pd.read_excel('Casos confirmados PR.xlsx', 'Casos confirmados', usecols='D:M')
casos_confirmados.columns = [ normalize_labels(x) for x in casos_confirmados.columns]
casos_confirmados['nome'] = casos_confirmados['nome'].apply(normalize_text)
casos_confirmados['idade'] = casos_confirmados['idade'].apply(normalize_number)
casos_confirmados['mun_resid'] = casos_confirmados['mun_resid'].apply(normalize_text)
casos_confirmados['mun_atend'] = casos_confirmados['mun_atend'].apply(normalize_text)
casos_confirmados['hash'] = casos_confirmados.apply(lambda row: (row['nome'] + str(row['idade']) + row['mun_resid']).replace(" ",""), axis=1)
casos_confirmados.dtypes

nome                   object
sexo                   object
idade                   int64
mun_resid              object
mun_atend              object
rs                    float64
laboratorio            object
dt_diag                object
comunicacao    datetime64[ns]
is                     object
hash                   object
dtype: object

In [13]:
print(f"obitos que estão nos casos {obitos_raw.loc[obitos_raw['hash'].isin(casos_confirmados['hash'])].shape[0]}")

obitos que estão nos casos 35


In [14]:
print(f"obitos que estão nos casos porém com um ano a menos {obitos_raw.loc[obitos_raw['hash_less'].isin(casos_confirmados['hash'])].shape[0]}")

obitos que estão nos casos porém com um ano a menos 0


In [15]:
print(f"obitos que estão nos casos porém com um ano a mais {obitos_raw.loc[obitos_raw['hash_more'].isin(casos_confirmados['hash'])].shape[0]}")

obitos que estão nos casos porém com um ano a mais 0


In [16]:
obitos_raw.columns

Index(['data_notificacao', 'id_ficha', 'nome_paciente', 'sexo', 'idade',
       'municipio_residencia', 'municipio_atendimento', 'rs', 'exame',
       'data_diagnostico', 'data_primeiro_sintomas', 'data_cura_obito', 'hash',
       'hash_less', 'hash_more'],
      dtype='object')

In [17]:
obitos_raw['rs'] = [ "" ] * len(obitos_raw)
novos_obitos = obitos_raw[['nome_paciente','sexo','idade','municipio_residencia', 'rs', 'data_cura_obito']]
novos_obitos.to_excel('novos_obitos.xlsx', index=False)